In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
# import keras as keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from os import listdir
import math

Differenze tra still, walking e running

In [ ]:
# relevant features
still = pd.read_csv('datasets/still_19_02_2023_17_28_18.csv')
walking = pd.read_csv('datasets/walking_19_02_2023_17_10_11.csv')
running = pd.read_csv('datasets/running_23_02_2023_10_48_27.csv')


fig, axs = plt.subplots(still.shape[1],3)

fig.set_figheight(50)
fig.set_figwidth(25)


for i in range (still.shape[1]):
    title = ''

    if i == 0:
        title = "Accelerometro Asse X"
    elif i == 1:
        title = "Accelerometro Asse Y"
    elif i == 2:
        title = "Accelerometro Asse Z"
    elif i == 3:
        title = "Giroscopio Asse X"
    elif i == 4:
        title = "Giroscopio Asse Y"
    elif i == 5:
        title = "Giroscopio Asse Z"
    elif i == 6:
        title = "Magnetometro Asse X"
    elif i == 7:
        title = "Magnetometro Asse Y"
    elif i == 8:
        title = "Magnetometro Asse Z"
    elif i == 9:
        title = "Temperatura (°C)"
    else:
        title = "Umidità (%)"

    axs[i,0].plot(np.arange(0, still.shape[0]), still[still.columns[i]])
    axs[i,0].set_title(f'{title} (Fermo)')

    axs[i,1].plot(np.arange(0, walking.shape[0]), walking[walking.columns[i]])
    axs[i,1].set_title(f'{title} (Camminata)')

    axs[i,2].plot(np.arange(0, running.shape[0]), running[running.columns[i]])
    axs[i,2].set_title(f'{title} (Corsa)')

Lettura dei dataset dalla cartella datasets

In [ ]:
# list of files inside datasets folder
files = listdir('datasets')

datasets = []

# for every file add a label
def divide_and_add_activity(name, df):
    # aggiungo la label dell'attvità
    if 'still' in name:
        df['activity'] = 'still'
    elif 'walking' in name:
        df['activity'] = 'walking'
    else:
        df['activity'] = 'running'
    return df

# add activity to the labels we are interested
labels = ['xa', 'ya', 'za', 'xg', 'yg', 'zg', 'activity']

for file in files:
    df = divide_and_add_activity(file, pd.read_csv(f'datasets/{file}'))
    df = df[labels]
    datasets.append(df)
    print("Dimensione dataset {} ".format(file), df.shape)


df_data = pd.concat(datasets)

print("Dimensione merged dataset", df_data.shape)
print(df_data)



Normalizzazione dei dati

In [ ]:
normalizer = MinMaxScaler()
num_columns = df_data.shape[1]

normalizer.fit(df_data.iloc[:, 0 : num_columns-1])

def minmax_normalizer(df):
    
    norm_df = df.copy()
    norm_column = normalizer.transform(df.iloc[:, 0 : num_columns-1])
    norm_df.iloc[:, 0 : num_columns-1] = norm_column

    return norm_df


def arduino_normalizer(df):
    norm_df = df.copy()
    labels = ['xa', 'ya', 'za', 'xg', 'yg', 'zg']
    norm_df[labels] = norm_df[labels] / 1000.0
    norm_df['xa'] = (norm_df['xa'] + 4.0) / 8.0
    norm_df['ya'] = (norm_df['ya'] + 4.0) / 8.0
    norm_df['za'] = (norm_df['za'] + 4.0) / 8.0
    norm_df['xg'] = (norm_df['xg'] + 2000.0) / 4000.0
    norm_df['yg'] = (norm_df['yg'] + 2000.0) / 4000.0
    norm_df['zg'] = (norm_df['zg'] + 2000.0) / 4000.0
    return norm_df


norm_df = arduino_normalizer(df_data)

# # visualizzazione dati grezzi e normalizzati dell'accelerometro
# fig, ((ax1, ax2),( ax3, ax4), (ax5, ax6)) = plt.subplots(3,2)
# fig.set_figheight(10)
# fig.set_figwidth(10)
# ax1.plot(np.arange(0, len(df_data.xa)), df_data.xa)
# ax1.set_title('Dati grezzi')
# ax3.plot(np.arange(0, len(df_data.ya)), df_data.ya)
# ax5.plot(np.arange(0, len(df_data.za)), df_data.za)

# ax2.plot(np.arange(0, len(norm_df.xa)), norm_df.xa)
# ax2.set_title('Dati normalizzati')
# ax4.plot(np.arange(0, len(norm_df.ya)), norm_df.ya)
# ax6.plot(np.arange(0, len(norm_df.za)), norm_df.za)


# norm_df

fig, axs = plt.subplots(6,2)

fig.set_figheight(25)
fig.set_figwidth(20)


for i in range (6):
    title = ''

    if i == 0:
        title = "Accelerometro Asse X"
    elif i == 1:
        title = "Accelerometro Asse Y"
    elif i == 2:
        title = "Accelerometro Asse Z"
    elif i == 3:
        title = "Giroscopio Asse X"
    elif i == 4:
        title = "Giroscopio Asse Y"
    elif i == 5:
        title = "Giroscopio Asse Z"
    elif i == 6:
        title = "Magnetometro Asse X"
    elif i == 7:
        title = "Magnetometro Asse Y"
    elif i == 8:
        title = "Magnetometro Asse Z"
    elif i == 9:
        title = "Temperatura (°C)"
    else:
        title = "Umidità (%)"

    axs[i,0].plot(np.arange(0, df_data.shape[0]), df_data[df_data.columns[i]])
    axs[i,0].set_title(f'{title} (Fermo)')

    axs[i,1].plot(np.arange(0, norm_df.shape[0]), norm_df[norm_df.columns[i]])
    axs[i,1].set_title(f'{title} (Camminata)')


Resize datasets

In [ ]:
def reshape_data(df, time_steps, step):
    Xs = []
    ys = []

    num_cols = df.shape[1]

    X = df.iloc[:, :num_cols - 1]
    y = df.iloc[:, num_cols - 1:]

    print(X.shape)

    for i in range(0, len(X) - time_steps, step):
        values = X.iloc[i : (i+time_steps)].values
        labels = y.iloc[i : i + time_steps]
        Xs.append(values)
        ys.append(labels.mode()['activity'][0])
    
    return np.array(Xs), np.array(ys).reshape(-1, 1)


X_data, y_data = reshape_data(norm_df, 20, 20)
print(X_data.shape)

Split test and train data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.33, random_state=42)

print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

OneHotEncoder per la y

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(y_train)

def encode_y(y):
    encoded_y = encoder.transform(y)
    return encoded_y

encoded_y_train = encode_y(y_train)
encoded_y_test = encode_y(y_test)


print(encoder.categories_)
# print(y_train)
print(pd.DataFrame(encoded_y_train, columns=encoder.categories_))

In [ ]:
# stampa delle dimensioni

print(X_train.shape)
print(X_test.shape)

print(encoded_y_train.shape)
print(encoded_y_test.shape)
encoded_y_train

# Modello

In [ ]:
model = Sequential()
model.add(Dense(16, input_shape=[X_train.shape[1], X_train.shape[2]], activation='relu'))
# model.add(Dropout(rate=0.1))
model.add(Dense(32, activation = 'relu'))
# model.add(Dropout(rate=0.1))
model.add(Dense(32, activation = 'relu'))
# model.add(Dropout(rate=0.1))
model.add(Flatten())
model.add(Dense(encoded_y_train.shape[1], activation = 'softmax'))
print(model.summary())


Modello - ottimizzatore e funzione di perdita

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

Early stopping per evitare overfitting

In [ ]:
early_stopping = EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    X_train, encoded_y_train,
    validation_data=(X_test, encoded_y_test),
    batch_size=3,
    epochs=50,
    callbacks=[early_stopping]
)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], 'r', label='Precisione sui dati di train')
plt.plot(history.history['val_accuracy'], 'b', label='Precisione sui dati di validazione')
plt.plot(history.history['loss'], 'r--', label='Perdita sui dati di train')
plt.plot(history.history['val_loss'], 'b--', label='Perdita sui dati di validazione')
plt.title('Precisione e Perdita del Modello')
plt.ylabel('Precisione e Perdita')
plt.xlabel('N. Epoca')
plt.ylim(0)
plt.legend()
plt.show()

Matrice per l'affidabilità della predizione sui dati di validazione

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
def show_confusion_matrix(validations, predictions):

    matrix = confusion_matrix(validations, predictions)
    plt.figure(figsize=(6, 4))
    sns.heatmap(matrix,
                cmap='coolwarm',
                linecolor='white',
                linewidths=1,
                xticklabels=['running', 'still', 'walking'],
                yticklabels=['running', 'still', 'walking'],
                annot=True,
                fmt='d')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

y_pred_test = model.predict(X_test)

# indice della massima probabilità
max_y_pred_test = np.argmax(y_pred_test, axis=1)

max_y_test = np.argmax(encoded_y_test, axis=1)

show_confusion_matrix(max_y_test, max_y_pred_test)

print(classification_report(max_y_test, max_y_pred_test))

In [ ]:
model.evaluate(X_test, encoded_y_test)

piccolo test con alcuni dati in input

In [ ]:
data_still = pd.read_csv('datasets/running_23_02_2023_10_48_27.csv')[labels[0:-1]]
data_norm = minmax_normalizer(data_still)

data_input = np.array([data_norm.iloc[7:27,:], data_norm.iloc[30:50,:]])
print(data_input.shape)
print(data_input)

res = model.predict(data_input)
print(res)
print(np.argmax(res[0]))

In [ ]:
data_still = pd.read_csv('datasets/still_20_02_2023_15_33_07.csv')[labels[0:-1]]

data_norm = minmax_normalizer(data_still)

data_input = []

count_still = 0
count_walking = 0
count_running = 0

# resize data in signals of 20 samples
for i in range(0, data_norm.shape[0]-20, 20):
    data_input.append(data_norm.iloc[i:(i+20), :])

input = np.array(data_input)
print(input.shape)

# classify the input
res = model.predict(input)
print(res)

# get the index of the biggest probability (0 = still, 1 = walking)
res = np.argmax(res, axis=1)
print(res)

for value in res:
    if value == 0:
        count_running += 1
    elif value == 1:
        count_still +=1
    else:
        count_walking += 1

print('Still: ', count_still)
print('Walking: ', count_walking)
print('Running: ', count_running)




Coverti il modello keras in un modello tensorflow lite

In [ ]:
model.save('activity_classifier_model')

# convert model to tensorflow lite
converter = tf.lite.TFLiteConverter.from_saved_model('activity_classifier_model')
tflite_model = converter.convert()

# Save the model to disk
with open("./activity_classifier.tflite", "wb") as f:
    f.write(tflite_model)


In [ ]:
data_vis = pd.read_csv('datasets/running_23_02_2023_10_48_27.csv')[labels[0:-1]]
data_norm = minmax_normalizer(data_still)

data_zero = np.zeros(20 * 6).reshape([1,20,6])

res = model.predict(data_zero)
print(res)

